In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "ab030-tracing"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.

from typing import cast
import uuid

from agentex.types import Task

TASK_ID = str(uuid.uuid4())[:8]

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="task/create",
    params={
        "name": f"{TASK_ID}-task",
        "params": {}
    }
)

task = cast(Task, rpc_response.result)
print(task)

Task(id='e52ad928-8d6f-431f-a865-c632dfd925bd', created_at=datetime.datetime(2025, 7, 27, 1, 16, 54, 390130, tzinfo=TzInfo(UTC)), name='4c21818b-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 27, 1, 16, 54, 390130, tzinfo=TzInfo(UTC)))


In [4]:
# Test non streaming response
from typing import cast
from agentex.types import Event

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="event/send",
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "task_id": task.id,
    }
)

event = cast(Event, rpc_response.result)
print(event)

Event(id='8777dbbf-5a48-40ee-9d95-5476223ac759', agent_id='d1771757-da6c-43a3-b6bd-63679456790e', sequence_id=212, task_id='e52ad928-8d6f-431f-a865-c632dfd925bd', content=TextContent(author='user', content='Hello what can you do?', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 27, 1, 16, 54, 847712, tzinfo=TzInfo(UTC)))


In [5]:
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=5,
)

╭───────────────────────── USER [07/27/2025 01:16:54] ─────────────────────────╮
│ Hello what can you do?                                                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 01:16:54] ─────────────────────────╮
│ Hello! I can assist you with a variety of tasks, including:                  │
│                                                                              │
│  1 Answering Questions: I can provide information on a wide range of topics, │
│    including history, science, technology, literature, and general           │
│    knowledge.                                                                │
│  2 Providing Explanations: I can explain concepts, summarize information, or │
│    help clarify difficult topics.                                            │
│  3 Writing Assistance: I can help you draft emails, essays, articles, or     │
│    creative writing pieces. I can also assist with proofreading and editing. │
│  4 Learning and Study Aid: I can help with study tips, summarizing           │
│    educational materials, and answering homework questions.                  │
│  5 Recommendations: I can suggest books, movies, recipes, or activities      │
│    based on your interests.                                                  │
│  6 Technical Help: I can provide assistance with basic troubleshooting for   │
│    software or digital tools.                                                │
│  7 Conversation: If you just want to chat or discuss a specific topic, I'm   │
│    here for that too!                                                        │
│                                                                              │
│ Let me know how I can assist you today!                                      │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 5 seconds - returning collected messages
